In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import langchain
from langchain.cache import InMemoryCache
import os
from dotenv import load_dotenv
langchain.cache = InMemoryCache()
load_dotenv()


True

In [2]:
chat = ChatOpenAI()

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate,
    PromptTemplate,
)

## Parse outputs

In [9]:

from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()


In [10]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [12]:
reply = "Hello, how are you?"

In [13]:
output_parser.parse(reply)

['Hello', 'how are you?']

In [19]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [16]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [17]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [22]:
chat_prompt.format_prompt(request='give me a list of 5 fruits',
                          format_instructions=output_parser.get_format_instructions()).to_messages()

[HumanMessage(content='give me a list of 5 fruits\nYour response should be a list of comma separated values, eg: `foo, bar, baz`', additional_kwargs={}, example=False)]

In [32]:
model_request = chat_prompt.format_prompt(request='write a poem about stick',
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [33]:
result = chat(model_request)

In [34]:
result.content

"Straight and slender, standing tall,\nA humble stick, beloved by all.\nIn forests deep or parks so green,\nIts presence there, a timeless scene.\n\nA wand of power in children's hands,\nUnleashing magic in enchanted lands.\nFrom knights and warriors to wizards' spell,\nA stick's transformation, stories tell.\n\nA faithful companion on nature's trail,\nGuiding our steps, without a fail.\nA makeshift bridge across the stream,\nA sturdy support for a cherished dream.\n\nIn winter's grasp, a tool to slide,\nOn frozen ponds, a joyous ride.\nFrom snowball fights to snowmen's charm,\nA stick's laughter, in frosty arms.\n\nA painter's brush, in an artist's style,\nCreating masterpieces that beguile.\nSwirling colors upon canvas grand,\nA stick's strokes, a masterpiece's hand.\n\nA game of fetch with a loyal hound,\nChasing a stick, both lost and found.\nA moment of joy in a wagging tail,\nA bond unspoken, beyond the trail.\n\nA crackling fire, on a starlit night,\nA stick's sacrifice, a warm 

In [35]:
output_parser.parse(result.content)

['Straight and slender',
 'standing tall,\nA humble stick',
 'beloved by all.\nIn forests deep or parks so green,\nIts presence there',
 "a timeless scene.\n\nA wand of power in children's hands,\nUnleashing magic in enchanted lands.\nFrom knights and warriors to wizards' spell,\nA stick's transformation",
 "stories tell.\n\nA faithful companion on nature's trail,\nGuiding our steps",
 "without a fail.\nA makeshift bridge across the stream,\nA sturdy support for a cherished dream.\n\nIn winter's grasp",
 'a tool to slide,\nOn frozen ponds',
 "a joyous ride.\nFrom snowball fights to snowmen's charm,\nA stick's laughter",
 "in frosty arms.\n\nA painter's brush",
 "in an artist's style,\nCreating masterpieces that beguile.\nSwirling colors upon canvas grand,\nA stick's strokes",
 "a masterpiece's hand.\n\nA game of fetch with a loyal hound,\nChasing a stick",
 'both lost and found.\nA moment of joy in a wagging tail,\nA bond unspoken',
 'beyond the trail.\n\nA crackling fire',
 "on a star

## Datetime Parser

In [36]:
from langchain.output_parsers import DatetimeOutputParser
output_parser = DatetimeOutputParser()

In [37]:
output_parser.get_format_instructions()

'Write a datetime string that matches the \n            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1758-09-03T11:44:41.010048Z, 0362-06-08T09:38:50.850969Z, 0825-05-02T01:03:38.607040Z'

In [38]:
template_text = "{request}\n{format_instructions}"

In [39]:
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

In [45]:
system_text = "you always reply to questions only in datetime patterns"
system_prompt = SystemMessagePromptTemplate.from_template(system_text)

In [51]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [52]:
query = chat_prompt.format_prompt(request='what is date India got independence',
                                  format_instructions=output_parser.get_format_instructions()).to_messages()

In [53]:
result = chat(query)

In [54]:
result.content

'1947-08-15T00:00:00.000000Z'

In [55]:
output_parser.parse(result.content)

datetime.datetime(1947, 8, 15, 0, 0)

### 2nd Approach

In [56]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [57]:
query = chat_prompt.format_prompt(request='what is date India got independence',
                                  format_instructions=output_parser.get_format_instructions()).to_messages()

In [58]:
result = chat(query)

In [59]:
result.content

'India got independence on August 15, 1947. Therefore, the datetime string for this event would be "1947-08-15T00:00:00.000000Z".'

In [60]:
from langchain.output_parsers import OutputFixingParser

In [61]:
output_parser

DatetimeOutputParser(format='%Y-%m-%dT%H:%M:%S.%fZ')

In [62]:
misformatted = result.content

In [64]:
outputfix = OutputFixingParser.from_llm(parser=output_parser, llm=chat)

In [65]:
outputfix.parse(misformatted)

datetime.datetime(1947, 8, 15, 0, 0)

### pydantic parsing

In [67]:
from langchain.output_parsers import PydanticOutputParser

In [69]:
from pydantic import BaseModel, Field

In [77]:
class Scientiest(BaseModel):
    name: str = Field(description='scientist')
    discoveries: list = Field(description='python list of discoveries')

In [78]:
parser = PydanticOutputParser(pydantic_object=Scientiest)

In [79]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}
```


In [80]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")

In [81]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [85]:
query = chat_prompt.format_prompt(request='tell me about a famous scientist',
                                format_instructions=parser.get_format_instructions()).to_messages()

In [86]:
result = chat(query, temperature=0)

In [87]:
result.content

'{\n  "name": "Albert Einstein",\n  "discoveries": ["Theory of Relativity", "Photoelectric Effect", "Brownian Motion"]\n}'

In [88]:
parser.parse(result.content)

Scientiest(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Brownian Motion'])

In [89]:
type(parser.parse(result.content))

__main__.Scientiest

## Saving and loading prompts

In [4]:
template_text = "Tell me about a {planent}"

In [5]:
prompt=PromptTemplate.from_template(template_text)

In [6]:
prompt

PromptTemplate(input_variables=['planent'], output_parser=None, partial_variables={}, template='Tell me about a {planent}', template_format='f-string', validate_template=True)

In [7]:
prompt.save('../data/planet_prompt.json')

In [8]:
from langchain.prompts import load_prompt

In [9]:
loaded_prompt = load_prompt('../data/planet_prompt.json')

In [10]:
loaded_prompt

PromptTemplate(input_variables=['planent'], output_parser=None, partial_variables={}, template='Tell me about a {planent}', template_format='f-string', validate_template=True)

## Data Connections

#### CSV loader

In [11]:
from langchain.document_loaders import CSVLoader

In [12]:
loader = CSVLoader('../data/penguins.csv')

In [13]:
data = loader.load()    

In [16]:
type(data[0])

langchain.schema.document.Document

In [17]:
data[0]

Document(page_content='species: Adelie\nisland: Torgersen\nbill_length_mm: 39.1\nbill_depth_mm: 18.7\nflipper_length_mm: 181\nbody_mass_g: 3750\nsex: MALE', metadata={'source': '../data/penguins.csv', 'row': 0})

In [20]:
print(data[0].page_content)

species: Adelie
island: Torgersen
bill_length_mm: 39.1
bill_depth_mm: 18.7
flipper_length_mm: 181
body_mass_g: 3750
sex: MALE


### HTML loader

In [24]:
from langchain.document_loaders import BSHTMLLoader

In [25]:
loader = BSHTMLLoader('../data/some_website.html')

In [26]:
data = loader.load()

In [31]:
data[0].page_content

'Heading 1'

### pdf loader

In [33]:
from langchain.document_loaders import PyPDFLoader

In [35]:
loader = PyPDFLoader('../data/SomeReport.pdf')

In [40]:
pages =  loader.load()

In [45]:
print(pages[0].page_content.replace('\n', ' '))

This is the first line PDF. This is the second line in the PDF. This is the third line in the PDF.


### Intergrations

In [46]:
from langchain.document_loaders import HNLoader

In [51]:
loader = HNLoader('https://news.ycombinator.com/item?id=37417027')

In [52]:
data = loader.load()

In [55]:
data[0].page_content

"noelwelsh 4 hours ago  \n             | next [–] \n\nLooks really nice. A few things that I think might make the presentation a bit clearer:1. Using polar coordinates makes the maths a lot cleaner than using Cartesian coordinates. However you then either need to explain polar coordinates, or you assume people remember polar coordinates from high school which is often not the case in my experience.2. I think the positioning of the points is a little bit opaque. I was expecting to see (r * cos theta, r * sin theta) and it took me by surprise to see an addition in there. Either just noting that stupid computer graphics libraries don't put the origin at the center, or adding that translation later (which is a chance to talk about function composition) might be beneficial.(I've written my own take on this same topic, starting at http://www.creativescala.org/creative-scala/polygons/  It will take a little bit more than 5 minutes to get through it :-)  It's really fun and you can do a huge a

In [56]:
human_prompt = HumanMessagePromptTemplate.from_template("Give a short Summary of the following hackernews comment\n{comment}")

In [57]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [69]:
query = chat_prompt.format_prompt(comment=data[0].page_content).to_messages()
""" iterate over all the data list and get the summary of each comment """

collection = []
for comment in data:
    query = chat_prompt.format_prompt(comment=comment.page_content).to_messages()
    result = chat(query)
    print(result.content)
    collection.append(result.content)


In this comment, the user noelwelsh provides feedback on a presentation, suggesting ways to make it clearer. They recommend using polar coordinates instead of Cartesian coordinates for cleaner math, but also mention the need to explain polar coordinates for those who may not be familiar with them. They also suggest clarifying the positioning of points and noting that computer graphics libraries don't always put the origin at the center. The user also mentions that they have written their own take on the same topic and provide a link to it.
The commenter suggests that the addition being discussed in the previous comments is intended to center the figures in the canvas.
The commenter appreciates the clear examples provided and suggests drawing inspiration from Daniel Shifman's book "Nature of code," particularly chapters 2 and 3.
The commenter expresses gratitude for the reference to a book and mentions that they will check it out.
The commenter is confused about how the concept or idea 

In [66]:
query

[HumanMessage(content="Give a short Summary of the following hackernews comment\nnoelwelsh 4 hours ago  \n             | next [–] \n\nLooks really nice. A few things that I think might make the presentation a bit clearer:1. Using polar coordinates makes the maths a lot cleaner than using Cartesian coordinates. However you then either need to explain polar coordinates, or you assume people remember polar coordinates from high school which is often not the case in my experience.2. I think the positioning of the points is a little bit opaque. I was expecting to see (r * cos theta, r * sin theta) and it took me by surprise to see an addition in there. Either just noting that stupid computer graphics libraries don't put the origin at the center, or adding that translation later (which is a chance to talk about function composition) might be beneficial.(I've written my own take on this same topic, starting at http://www.creativescala.org/creative-scala/polygons/  It will take a little bit mo

In [67]:
result = chat(query)

In [68]:
print(result.content)

The commenter believes that the presentation could be improved by using polar coordinates instead of Cartesian coordinates, but acknowledges that not everyone may be familiar with polar coordinates. They also suggest making the positioning of the points clearer, either by explaining that computer graphics libraries don't put the origin at the center or by adding a translation later. The commenter also mentions that they have written their own take on the topic and provide a link to it.
